# 01 - Intro to PyBullet

## 1 - Getting Started

Starting the PyBullet simulator with GUI.

In [ ]:
import pybullet as p
physicsClient = p.connect(p.GUI)

Configuring a few options to make the UI cleaner:

In [ ]:
p.configureDebugVisualizer(p.COV_ENABLE_RGB_BUFFER_PREVIEW, False, physicsClientId=physicsClient)
p.configureDebugVisualizer(p.COV_ENABLE_DEPTH_BUFFER_PREVIEW, False, physicsClientId=physicsClient)
p.configureDebugVisualizer(p.COV_ENABLE_SEGMENTATION_MARK_PREVIEW, False, physicsClientId=physicsClient)
p.configureDebugVisualizer(p.COV_ENABLE_SHADOWS, True, physicsClientId=physicsClient)

Loading an URDF file.

In [ ]:
import planning
robot_id = p.loadURDF(planning.ur5e, useFixedBase=True)
robot_id

## 2 - pybullet_planning
[pybullet_planning](https://github.com/yijiangh/pybullet_planning) is a package that provides a few convenience functions for PyBullet.

In [ ]:
import pybullet_planning as pp

In [ ]:
joint_ids = pp.get_movable_joints(robot_id)
joint_ids

In [ ]:
pp.get_joint_names(robot_id, joint_ids)

In [ ]:
link_ids = pp.get_all_links(robot_id)

for link_id in link_ids:
    print(f"{link_id:2} {pp.get_link_name(robot_id, link_id)}")


In [ ]:
for link_id in link_ids:
    color = (1, 1, 1, 0.5)
    p.changeVisualShape(robot_id, link_id, shapeIndex=-1, rgbaColor=color)

In [ ]:
GEOM_TYPES = {
    p.GEOM_SPHERE: "sphere",
    p.GEOM_BOX: "box",
    p.GEOM_CYLINDER: "cylinder",
    p.GEOM_MESH: "mesh",
    p.GEOM_PLANE: "plane",
    p.GEOM_CAPSULE: "capsule",
    p.GEOM_HEIGHTFIELD: "heightfield",
}

collision_link_ids = []

for link_id in link_ids:
    collisionShapeData = p.getCollisionShapeData(robot_id, link_id)
    if not collisionShapeData:
        continue

    collision_link_ids.append(link_id)
    # Print info
    link_name = pp.get_link_name(robot_id, link_id)
    print(f"{link_name:20} {GEOM_TYPES[collisionShapeData[0][2]]}")
    # p.addUserDebugText(f"{link_id:2} {pp.get_link_name(robot_id, link_id)}", pp.get_link_pose(robot_id, link_id)[0], textColorRGB=[1, 1, 1], textSize=1.5, lifeTime=0)

collision_link_ids

In [ ]:
from itertools import product
example = ['a', 'b', 'c']
example_pairs = list(product(example, example))
print(example_pairs)

In [ ]:
collision_pairs = list(product(collision_link_ids, collision_link_ids))
len(collision_pairs)

In [ ]:
def remove_link_self_pairs(pairs):
    return [pair for pair in pairs if pair[0] != pair[1]]


def remove_equivalent_pairs(pairs):
    # collision checking is commutative so we only need to check one direction
    return [pair for pair in pairs if pair[0] <= pair[1]]

def remove_parent_child_pairs(pairs):
    return [pair for pair in pairs if not pp.are_links_adjacent(robot_id, *pair)]

collision_pairs = remove_link_self_pairs(collision_pairs)
collision_pairs = remove_equivalent_pairs(collision_pairs)
collision_pairs = remove_parent_child_pairs(collision_pairs)
len(collision_pairs) 

In [ ]:
for i, pair in enumerate(collision_pairs):
    linkA_id, linkB_id = pair
    print(f"{i:2} {pp.get_link_name(robot_id, linkA_id):20} {pp.get_link_name(robot_id, linkB_id)}")

In [ ]:
joint_angles = [-0.39215112,  2.46158236,  2.91327904, -0.73235854,  1.83296247,  0.18155214]
pp.set_joint_positions(robot_id, joint_ids, joint_angles)


In [ ]:
def get_colliding_pairs(pairs):
    colliding_pairs = []
    # p.performCollisionDetection() # not sure if this is required?
    for linkA_id, linkB_id in pairs:
        points =p.getClosestPoints(bodyA=robot_id, bodyB=robot_id, distance=0.0, linkIndexA=linkA_id, linkIndexB=linkB_id)

        if len(points) > 0:
            colliding_pairs.append((linkA_id, linkB_id))
    return colliding_pairs
        


colliding_pairs = get_colliding_pairs(collision_pairs)
for linkA_id, linkB_id in colliding_pairs:    
    print(f"{pp.get_link_name(robot_id, linkA_id):20} {pp.get_link_name(robot_id, linkB_id)}")
    colliding_color = (1, 0, 0, 0.2)
    p.changeVisualShape(robot_id, linkA_id, shapeIndex=-1, rgbaColor=colliding_color)
    p.changeVisualShape(robot_id, linkB_id, shapeIndex=-1, rgbaColor=colliding_color)

In [ ]:
# collision_fn = pp.get_collision_fn(
#     robot_id, ik_joints, obstacles=[], attachments=[], self_collisions=True, disabled_collisions=self_collision_links
# )

# performCollisionDetection
# setCollisionFilterGroupMask

In [ ]:
import numpy as np
# random_joints = np.random.uniform(-np.pi, np.pi, len(joints))
# colliding = collision_fn(random_joints)
# colliding

In [ ]:
np.random.seed(0)

In [ ]:
import time
random_joints_start = np.random.uniform(-np.pi, np.pi, len(joint_ids))
random_joints_end = np.random.uniform(-np.pi, np.pi, len(joint_ids))

print(random_joints_end)

n_steps = 200
for i in range(n_steps):
    alpha = i / (n_steps - 1)
    joint_angles = (1 - alpha) * random_joints_start + alpha * random_joints_end
    pp.set_joint_positions(robot_id, joint_ids, joint_angles)
    time.sleep(0.01)


In [ ]:
p.disconnect()